In [1]:
!pip install simpletransformers


     |████████████████████████████████| 219 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 49.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 37.6 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 31.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 43.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 48.1 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 50.7 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 39.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 6.6 MB/s  eta 0:00:01
     |████████████████████████████████| 159 kB 37.0 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 29.6 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 36.1 MB/s eta 0:00:01
     |███████████████████████████████

  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=a756b53dea9a8788b84cc61408334d8defad1fc469d840deb28ffc74bfbf86d2
  Stored in directory: /home/kgsc8254/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=9d9bf478d3ba338455a13b563f1d05ac31eaffd33aa26e055b7aebadb6d4e96c
  Stored in directory: /home/kgsc8254/.cache/pip/wheels/7b/78/f4/27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=d1368c61baf73ba5b126c36c30e4a3b99e7645d38b4df13cce7129ed5e1da30e
  Stored in directory: /home/kgsc8254/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6487 sha256=243158fc9b7f04f16aa13756880ac3d11a772aec1e867b96816ba93245ba81ab
  Stored in directory: /home/

# SimpleTransformers 
## + Hyperparam Tuning  + k-fold CV

In [2]:
import os, re, string
import random

import numpy as np
import pandas as pd
import sklearn

import torch

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'torch'

In [ ]:
# 토치 깔기 !https://pytorch.org/get-started/locally/

In [3]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

NameError: name 'torch' is not defined

In [6]:
for dirname, _, filenames in os.walk('../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/glove.6B.50d.txt
../data/glove.6B.100d.txt
../data/glove.6B.200d.txt
../data/test_x.csv
../data/glove.6B.300d.txt
../data/sample_submission.csv
../data/train.csv


In [8]:
train_data = pd.read_csv('../data/train.csv')
train_data = train_data[['text', 'author']]
train_data

,text,author
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...
54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,"I told my plan to the captain, and between us ...",4
54876,"""Your sincere well-wisher, friend, and sister...",1
54877,“Then you wanted me to lend you money?”,3


## 모델 기본값 확인 및 조정

In [9]:
# Using 'bert-large-uncased' here. For a list of other models, please refer to 
# https://github.com/ThilinaRajapakse/simpletransformers/#current-pretrained-models 
bert_uncased = ClassificationModel('bert', 'bert-large-uncased') 

# Print out all the default arguments for reference
bert_uncased.args

NameError: name 'ClassificationModel' is not defined

In [ ]:
# This is where we can tweak based on the default arguments above
custom_args = {'fp16': False, # not using mixed precision 
               'train_batch_size': 4, # default is 8
               'gradient_accumulation_steps': 2,
               'do_lower_case': True,
               'learning_rate': 1e-05, # using lower learning rate
               'overwrite_output_dir': True, # important for CV
               'num_train_epochs': 2} # default is 1

## 5-Fold CV

In [ ]:
n=5
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
results = []

for train_index, val_index in kf.split(train_data):
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    
    model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
    model.train_model(train_df)
    result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=sklearn.metrics.accuracy_score)
    print(result['acc'])
    results.append(result['acc'])

In [ ]:
for i, result in enumerate(results, 1):
    print(f"Fold-{i}: {result}")
    
print(f"{n}-fold CV accuracy result: Mean: {np.mean(results)} Standard deviation:{np.std(results)}")

## Full Training

In [ ]:
model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
model.train_model(train_data)

In [ ]:
p_val,_=model.predict(train_data['text'])

In [10]:
print(f'Accuracy (CV): {accuracy_score(pd.get_dummies(train_data['author']), np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(train_data['author']), p_val):8.4f}')

SyntaxError: invalid syntax (<ipython-input-10-0aea489aa523>, line 1)

In [12]:
print(f'Accuracy (CV): {accuracy_score(train_data['author'].values, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(train_data['author'].values), p_val):8.4f}')

SyntaxError: invalid syntax (<ipython-input-12-59ffc4169e8f>, line 2)

## Predict

In [ ]:
test_data = pd.read_csv('../data/test_x.csv')
predictions, raw_outputs = model.predict(test_data['text'])

sample_submission = pd.read_csv("/data/sample_submission.csv")
sample_submission[sample_submission.columns] = predictions
sample_submission.to_csv("Trans_HyperKfold.csv", index=False)